In [1]:
import json
import numpy as np
import os
from scipy.linalg import expm, logm
import sys

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp

In [2]:
def inner_product(vec1, vec2):
    """Compute the inner product of two complex vectors."""
    return np.vdot(vec1, vec2)  # vdot handles complex conjugation automatically

# Define the qubit states as complex numpy arrays
psi = (1 / np.sqrt(6)) * np.array([2, 1 + 1j])
phi = (1 / np.sqrt(10)) * np.array([1 - 1j, 3])

# Compute the inner product
inner_prod = inner_product(psi, phi)

# Display the result
print("Inner product:", inner_prod)

Inner product: (0.6454972243679029-0.6454972243679029j)


In [3]:
def encode_to_qubit(arr):
    """Encodes a real array of size 4 into a qubit state using amplitude encoding."""
    assert len(arr) == 4, "Array must have exactly 4 elements"
    complex_vec = np.array([arr[0] + 1j * arr[1], arr[2] + 1j * arr[3]])
    norm = np.linalg.norm(complex_vec)
    return complex_vec / norm if norm != 0 else complex_vec

# Define two real arrays of size 4
real_array1 = np.array([2, 1, 3, -1])
real_array2 = np.array([1, -1, 4, 2])

# Encode them into qubit states
psi = encode_to_qubit(real_array1)
phi = encode_to_qubit(real_array2)

# Compute the inner product
inner_prod = inner_product(psi, phi)

# Compute the magnitude of the inner product
inner_magnitude = np.abs(inner_prod)

# Display the results
print("Inner product:", inner_prod)
print("Magnitude of inner product:", inner_magnitude)


Inner product: (0.6055300708194982+0.38533731779422614j)
Magnitude of inner product: 0.7177405625652732


In [4]:
# Define the file to store embeddings
EMBEDDINGS_FILE = "embeddings.json"

def load_embeddings():
    """Loads stored embeddings from a JSON file if it exists."""
    if os.path.exists(EMBEDDINGS_FILE):
        with open(EMBEDDINGS_FILE, "r") as f:
            return json.load(f)
    return {}

def get_embeddings(word, stored_embeddings):
    """Retrieves embedding from cache or fetches it from OpenAI if not cached."""
    if word in stored_embeddings:
        print(f"Loaded cached embedding for '{word}'")
        return stored_embeddings[word]

def encode_to_qubit(arr):
    """Encodes a real array of any even size into a qubit state using amplitude encoding."""
    assert len(arr) % 2 == 0, "Array length must be even"
    complex_vec = np.array([arr[i] + 1j * arr[i+1] for i in range(0, len(arr), 2)])
    norm = np.linalg.norm(complex_vec)
    return complex_vec / norm if norm != 0 else complex_vec

words = ["cat", "dog", "car"]

# Load existing embeddings
stored_embeddings = load_embeddings()

# Retrieve embeddings (from file or OpenAI)
embeddings = {word: get_embeddings(word, stored_embeddings) for word in words}

# Extract individual vectors correctly
cat_vec = embeddings["cat"]
dog_vec = embeddings["dog"]
car_vec = embeddings["car"]

# Encode them into qubit states
cat_state = encode_to_qubit(cat_vec)
dog_state = encode_to_qubit(dog_vec)
car_state = encode_to_qubit(car_vec)

# Compute the inner product
cat_dog_inner_prod = inner_product(cat_state, dog_state)
cat_car_inner_prod = inner_product(cat_state, car_state)

# Compute the magnitude of the inner product
cat_dog_magnitude = np.abs(cat_dog_inner_prod)
cat_car_magnitude = np.abs(cat_car_inner_prod)

# Display the results
print("cat/dog Inner product:", cat_dog_inner_prod)
print("cat/car Inner product:", cat_car_inner_prod)

print("cat/dog Magnitude of inner product:", cat_dog_magnitude)
print("cat/car Magnitude of inner product:", cat_car_magnitude)


Loaded cached embedding for 'cat'
Loaded cached embedding for 'dog'
Loaded cached embedding for 'car'
cat/dog Inner product: (0.862957645542653-0.016694274937911695j)
cat/car Inner product: (0.845207835908355-0.010998340229826991j)
cat/dog Magnitude of inner product: 0.8631191092869059
cat/car Magnitude of inner product: 0.8452793913072149


In [20]:
import numpy as np

def inner_product(vec1, vec2):
    """Compute the inner product of two complex vectors."""
    return np.vdot(vec1, vec2)

def encode_to_qubit(arr):
    """
    Direct amplitude encoding:
    Groups pairs of reals into complex numbers and normalizes.
    """
    complex_vec = np.array([arr[i] + 1j * arr[i+1] for i in range(0, len(arr), 2)])
    norm = np.linalg.norm(complex_vec)
    return complex_vec / norm if norm != 0 else complex_vec

def rotation_matrix_y(theta):
    """Construct an R_y(theta) rotation matrix."""
    return np.array([
        [np.cos(theta/2), -np.sin(theta/2)],
        [np.sin(theta/2),  np.cos(theta/2)]
    ], dtype=complex)

def rotation_matrix_z(phi):
    """Construct an R_z(phi) phase shift matrix."""
    return np.array([
        [np.exp(-1j * phi/2), 0],
        [0, np.exp(1j * phi/2)]
    ], dtype=complex)

def recursive_state_preparation(psi):
    """
    Recursively prepares a state |psi> (with 2^n amplitudes) from |0...0>
    by splitting the state into left and right halves. At each step,
    we compute the rotation angle theta and relative phase phi so that
      cos(theta/2) = norm(left branch)
      sin(theta/2) = norm(right branch)
    and we force the left branch amplitude to be real and nonnegative.
    
    The unitary on the current qubit is defined as:
      U' = exp(i*phi/2)*R_z(phi)*R_y(theta)
    so that U'|0> = [cos(theta/2), sin(theta/2)*exp(i*phi)].
    """
    # Force global phase so that psi[0] is real and nonnegative.
    if psi[0] != 0:
        psi = psi * np.exp(-1j * np.angle(psi[0]))
    
    n = int(np.log2(len(psi)))
    if n == 0:
        return psi

    half = len(psi) // 2
    norm_left = np.linalg.norm(psi[:half])
    norm_right = np.linalg.norm(psi[half:])
    
    # Compute rotation angle: cos(theta/2)=norm_left, sin(theta/2)=norm_right.
    theta = 2 * np.arctan2(norm_right, norm_left)
    
    # Set the relative phase from the right branch.
    phi = np.angle(psi[half]) if (norm_left > 0 and norm_right > 0) else 0.0

    # Build the corrected single-qubit unitary.
    U = np.exp(1j * phi/2) * (rotation_matrix_z(phi) @ rotation_matrix_y(theta))
    qubit_state = U @ np.array([1, 0], dtype=complex)
    
    # Recursively prepare the left and right branches.
    if norm_left > 0:
        psi_left = psi[:half] / norm_left
        left_state = recursive_state_preparation(psi_left)
    else:
        left_state = np.zeros(half, dtype=complex)
    if norm_right > 0:
        psi_right = psi[half:] / norm_right
        right_state = recursive_state_preparation(psi_right)
    else:
        right_state = np.zeros(half, dtype=complex)
    
    return np.concatenate([qubit_state[0] * left_state,
                           qubit_state[1] * right_state])

def amplitude_encoding_real_hardware(vector):
    """
    Simulates amplitude encoding using only unitary operations
    (as on real quantum hardware) by recursively preparing the state.
    """
    # Convert the real vector into complex amplitudes.
    complex_vec = np.array([vector[i] + 1j * vector[i+1] for i in range(0, len(vector), 2)])
    norm = np.linalg.norm(complex_vec)
    if norm == 0:
        raise ValueError("Vector cannot be zero.")
    target_state = complex_vec / norm

    # Force global phase so that the first amplitude is real and nonnegative.
    if target_state[0] != 0:
        target_state = target_state * np.exp(-1j * np.angle(target_state[0]))
    
    # Prepare the state recursively.
    return recursive_state_preparation(target_state)

# Example vector for encoding (must have an even number of elements).
vector = np.array([0.5, 0.5, 0.5, 0.5, 0.25, 0.75, 0.1, 0.9])

# Encode the state using both methods.
encoded_state_direct = encode_to_qubit(vector)
encoded_state_hardware = amplitude_encoding_real_hardware(vector)

# Optionally, force the same global phase on both states.
def fix_global_phase(state):
    if state[0] != 0:
        return state * np.exp(-1j * np.angle(state[0]))
    return state

encoded_state_direct = fix_global_phase(encoded_state_direct)
encoded_state_hardware = fix_global_phase(encoded_state_hardware)

# Compute the inner product between the two states.
comparison_inner_prod = inner_product(encoded_state_direct, encoded_state_hardware)
comparison_magnitude = np.abs(comparison_inner_prod)

print("Encoded Quantum State (Direct Method):", encoded_state_direct)
print("Encoded Quantum State (Hardware-Simulated):", encoded_state_hardware)
print("Inner Product Magnitude Between Methods:", comparison_magnitude)

if np.isclose(comparison_magnitude, 1.0, atol=1e-6):
    print("The two methods produce equivalent quantum states (up to a global phase).")
else:
    print("The two methods produce different quantum states.")


Encoded Quantum State (Direct Method): [0.45221563+3.89009787e-17j 0.45221563+3.89009787e-17j
 0.45221563+2.26107816e-01j 0.45221563+3.61772505e-01j]
Encoded Quantum State (Hardware-Simulated): [0.45221563+6.53691246e-34j 0.45221563-7.74608296e-34j
 0.45221563+2.26107816e-01j 0.45221563+3.61772505e-01j]
Inner Product Magnitude Between Methods: 1.0
The two methods produce equivalent quantum states (up to a global phase).


In [21]:
import json
import os
import numpy as np
from scipy.linalg import expm

def load_embeddings():
    """Loads stored embeddings from a JSON file if it exists."""
    EMBEDDINGS_FILE = "embeddings.json"
    if os.path.exists(EMBEDDINGS_FILE):
        with open(EMBEDDINGS_FILE, "r") as f:
            return json.load(f)
    return {}

def get_embedding(word, stored_embeddings):
    """Retrieves the embedding for the given word from the cache."""
    if word in stored_embeddings:
        print(f"Loaded cached embedding for '{word}'")
        return stored_embeddings[word]
    return None

def inner_product(vec1, vec2):
    """Compute the inner product of two complex vectors."""
    return np.vdot(vec1, vec2)

def encode_to_qubit(arr):
    """
    Direct amplitude encoding:
    Groups pairs of reals into complex numbers and normalizes.
    """
    # Ensure the vector has an even number of elements.
    assert len(arr) % 2 == 0, "Array length must be even"
    complex_vec = np.array([arr[i] + 1j * arr[i+1] for i in range(0, len(arr), 2)])
    norm = np.linalg.norm(complex_vec)
    return complex_vec / norm if norm != 0 else complex_vec

def rotation_matrix_y(theta):
    """Construct an R_y(theta) rotation matrix."""
    return np.array([
        [np.cos(theta/2), -np.sin(theta/2)],
        [np.sin(theta/2),  np.cos(theta/2)]
    ], dtype=complex)

def rotation_matrix_z(phi):
    """Construct an R_z(phi) phase shift matrix."""
    return np.array([
        [np.exp(-1j * phi/2), 0],
        [0, np.exp(1j * phi/2)]
    ], dtype=complex)

def recursive_state_preparation(psi):
    """
    Recursively prepares a state |psi> (with 2^n amplitudes) from |0...0>
    by splitting the state into left and right halves. At each step,
    we compute the rotation angle theta and relative phase phi so that
      cos(theta/2) = norm(left branch)
      sin(theta/2) = norm(right branch)
    and we force the left branch amplitude to be real and nonnegative.
    
    The unitary on the current qubit is defined as:
      U' = exp(i*phi/2)*R_z(phi)*R_y(theta)
    so that U'|0> = [cos(theta/2), sin(theta/2)*exp(i*phi)].
    """
    # Force global phase so that psi[0] is real and nonnegative.
    if psi[0] != 0:
        psi = psi * np.exp(-1j * np.angle(psi[0]))
    
    n = int(np.log2(len(psi)))
    if n == 0:
        return psi

    half = len(psi) // 2
    norm_left = np.linalg.norm(psi[:half])
    norm_right = np.linalg.norm(psi[half:])
    
    # Compute rotation angle: cos(theta/2)=norm_left, sin(theta/2)=norm_right.
    theta = 2 * np.arctan2(norm_right, norm_left)
    
    # Set the relative phase from the right branch.
    phi = np.angle(psi[half]) if (norm_left > 0 and norm_right > 0) else 0.0

    # Build the corrected single-qubit unitary.
    U = np.exp(1j * phi/2) * (rotation_matrix_z(phi) @ rotation_matrix_y(theta))
    qubit_state = U @ np.array([1, 0], dtype=complex)
    
    # Recursively prepare the left and right branches.
    if norm_left > 0:
        psi_left = psi[:half] / norm_left
        left_state = recursive_state_preparation(psi_left)
    else:
        left_state = np.zeros(half, dtype=complex)
    if norm_right > 0:
        psi_right = psi[half:] / norm_right
        right_state = recursive_state_preparation(psi_right)
    else:
        right_state = np.zeros(half, dtype=complex)
    
    return np.concatenate([qubit_state[0] * left_state,
                           qubit_state[1] * right_state])

def amplitude_encoding_real_hardware(vector):
    """
    Simulates amplitude encoding using only unitary operations
    (as on real quantum hardware) by recursively preparing the state.
    """
    # Convert the real vector into complex amplitudes.
    complex_vec = np.array([vector[i] + 1j * vector[i+1] for i in range(0, len(vector), 2)])
    norm = np.linalg.norm(complex_vec)
    if norm == 0:
        raise ValueError("Vector cannot be zero.")
    target_state = complex_vec / norm

    # Force global phase so that the first amplitude is real and nonnegative.
    if target_state[0] != 0:
        target_state = target_state * np.exp(-1j * np.angle(target_state[0]))
    
    # Prepare the state recursively.
    return recursive_state_preparation(target_state)

def fix_global_phase(state):
    """Force the first amplitude to be real and nonnegative."""
    if state[0] != 0:
        return state * np.exp(-1j * np.angle(state[0]))
    return state

# --- Use the 'cat' embedding ---

stored_embeddings = load_embeddings()
cat_vec = get_embedding("cat", stored_embeddings)
if cat_vec is None:
    print("No embedding found for 'cat'")
else:
    cat_vec = np.array(cat_vec)
    
    # Direct amplitude encoding on cat_vec.
    cat_state_direct = fix_global_phase(encode_to_qubit(cat_vec))
    
    # Simulated hardware state preparation on cat_vec.
    cat_state_hardware = fix_global_phase(amplitude_encoding_real_hardware(cat_vec))
    
    # Optionally, force the same global phase.
    phase_offset = np.angle(inner_product(cat_state_direct, cat_state_hardware))
    cat_state_hardware *= np.exp(-1j * phase_offset)
    
    # Compute the inner product.
    comparison_inner_prod = inner_product(cat_state_direct, cat_state_hardware)
    comparison_magnitude = np.abs(comparison_inner_prod)
    
    #print("Encoded Quantum State (Direct Method):", cat_state_direct)
    #print("Encoded Quantum State (Hardware-Simulated):", cat_state_hardware)
    print("Inner Product Magnitude Between Methods:", comparison_magnitude)
    
    if np.isclose(comparison_magnitude, 1.0, atol=1e-6):
        print("The two methods produce equivalent quantum states (up to a global phase).")
    else:
        print("The two methods produce different quantum states.")


Loaded cached embedding for 'cat'
Inner Product Magnitude Between Methods: 0.9999999999999996
The two methods produce equivalent quantum states (up to a global phase).


In [22]:
cat_state_hardware = fix_global_phase(amplitude_encoding_real_hardware(cat_vec))
dog_state_hardware = fix_global_phase(amplitude_encoding_real_hardware(dog_vec))
car_state_hardware = fix_global_phase(amplitude_encoding_real_hardware(car_vec))

cat_dog_inner_prod = inner_product(cat_state_hardware, dog_state_hardware)
cat_dog_magnitude = np.abs(cat_dog_inner_prod)

cat_car_inner_prod = inner_product(cat_state_hardware, car_state_hardware)
cat_car_magnitude = np.abs(cat_car_inner_prod)

# Display the results
print("cat/dog Inner product:", cat_dog_inner_prod)
print("cat/car Inner product:", cat_car_inner_prod)

print("cat/dog Magnitude of inner product:", cat_dog_magnitude)
print("cat/car Magnitude of inner product:", cat_car_magnitude)

cat/dog Inner product: (0.8436195142419958-0.1824300194768403j)
cat/car Inner product: (0.8434195949134784-0.056041380110405936j)
cat/dog Magnitude of inner product: 0.8631191092869056
cat/car Magnitude of inner product: 0.8452793913072145


In [5]:
def normalize(vector):
    """Normalize a complex vector."""
    norm = np.linalg.norm(vector)
    if norm == 0:
        raise ValueError("Zero vector cannot be normalized.")
    return vector / norm

def pair_real_to_complex(vector):
    """Convert a real-valued vector into a complex-valued vector by pairing elements."""
    if len(vector) % 2 != 0:
        raise ValueError("Vector length must be even to pair into complex numbers.")
    complex_vector = np.array([vector[i] + 1j * vector[i+1] for i in range(0, len(vector), 2)])
    return complex_vector

def pad_vector(vector, target_size):
    """Pad the vector with zeros to the nearest power of 2."""
    if len(vector) > target_size:
        raise ValueError("Vector is larger than the target size.")
    padded_vector = np.zeros(target_size, dtype=complex)
    padded_vector[:len(vector)] = vector  # Copy original values
    return padded_vector

def get_rotation_angles(state):
    """Compute rotation angles theta and phi from state vector."""
    norm_left = np.linalg.norm(state[:len(state)//2])
    norm_right = np.linalg.norm(state[len(state)//2:])
    
    theta = 2 * np.arctan2(norm_right, norm_left)  # Ry rotation
    phi = np.angle(state[len(state)//2]) if (norm_left > 0 and norm_right > 0) else 0.0  # Rz phase shift
    
    return theta, phi

def apply_recursive_state_preparation(qc, qubits, state, control=None):
    n = len(state)
    if n == 1:
        return

    half = n // 2
    theta, phi = get_rotation_angles(state)
    target = qubits[0]

    # If there's no control, apply the rotation directly
    if control is None:
        qc.rz(phi, target)
        qc.ry(theta, target)
    else:
        # Apply controlled rotations (these gates are not built-in; one must construct them)
        qc.crz(phi, control, target)
        qc.cry(theta, control, target)

    # For each branch, the next set of qubits will be used.
    # Here you would prepare left and right branches conditionally.
    # This is a nontrivial step and typically involves multi-controlled rotations.
    left_state = normalize(state[:half]) if np.linalg.norm(state[:half]) > 0 else np.zeros(half, dtype=complex)
    right_state = normalize(state[half:]) if np.linalg.norm(state[half:]) > 0 else np.zeros(half, dtype=complex)

    # Recurse: use qubits[1:] for the sub-circuits, with target qubit as the new control.
    if len(qubits) > 1:
        apply_recursive_state_preparation(qc, qubits[1:], left_state, control=target)
        apply_recursive_state_preparation(qc, qubits[1:], right_state, control=target)


def prepare_amplitude_encoding(vector, state_vec=True):
    """Create a Qiskit quantum circuit that prepares the amplitude-encoded state."""
    vector = np.array(vector)
    
    # Step 1: Convert reals to complex numbers
    complex_vector = pair_real_to_complex(vector)
    
    # Step 2: Compute the nearest power of 2
    num_qubits = int(np.ceil(np.log2(len(complex_vector))))
    target_size = 2 ** num_qubits  # Next power of 2 (1024 for 768 complex values)
    
    # Step 3: Pad and normalize the vector
    padded_vector = pad_vector(complex_vector, target_size)
    normalized_vector = normalize(padded_vector)
    
    # Step 4: Create quantum circuit
    qc = QuantumCircuit(num_qubits)
    qubits = list(range(num_qubits))

    apply_recursive_state_preparation(qc, qubits, normalized_vector)

    # Step 5: **Explicitly save the statevector (Fix for Qiskit 1+)**
    if state_vec:
        qc.save_statevector()

    return qc

qc = prepare_amplitude_encoding(cat_vec)

# --- Simulating the state ---
# backend = Aer.get_backend('statevector_simulator')
# job = execute(qc, backend)
# result = job.result()
# state = result.get_statevector()


In [6]:
def get_qiskit_state(vec):
    qc = prepare_amplitude_encoding(vec)
    # --- Running the circuit on AerSimulator (Statevector Mode) ---
    simulator = AerSimulator(method='statevector')
    
    # Transpile for the AerSimulator
    compiled_circuit = transpile(qc, simulator)
    
    # Run the circuit and retrieve the statevector
    job = simulator.run(compiled_circuit)
    result = job.result()
    return np.asarray(result.get_statevector())[::-1]

psi_cat = get_qiskit_state(cat_vec)
psi_dog = get_qiskit_state(dog_vec)
psi_car = get_qiskit_state(car_vec)

cat_dog_inner_prod = inner_product(psi_cat, psi_dog)
cat_dog_magnitude = np.abs(cat_dog_inner_prod)

cat_car_inner_prod = inner_product(psi_cat, psi_car)
cat_car_magnitude = np.abs(cat_car_inner_prod)

# Display the results
print("cat/dog Inner product:", cat_dog_inner_prod)
print("cat/car Inner product:", cat_car_inner_prod)

print("cat/dog Magnitude of inner product:", cat_dog_magnitude)
print("cat/car Magnitude of inner product:", cat_car_magnitude)

cat/dog Inner product: (0.8858226596196912+0.39431826432717776j)
cat/car Inner product: (0.8157520786024126+0.45544796263063625j)
cat/dog Magnitude of inner product: 0.9696229565546092
cat/car Magnitude of inner product: 0.934282773259014


In [7]:
cat_cat_inner_prod = inner_product(psi_cat, cat_state_hardware)
cat_cat_magnitude = np.abs(cat_cat_inner_prod)

print("cat/cat Inner product:", cat_cat_inner_prod)
print("cat/cat Magnitude of inner product:", cat_cat_magnitude)

NameError: name 'cat_state_hardware' is not defined

In [101]:
def get_hamiltonian(circuit):
    """Compute the Hamiltonian H such that U = exp(-iH) for a given quantum circuit."""
    # Get the unitary matrix representation of the circuit
    U = Operator(circuit).data  # Extract the matrix

    # Compute the matrix logarithm to get the Hamiltonian
    H = -1j * logm(U)

    return H

vector = np.random.rand(1536)
qc = prepare_amplitude_encoding(vector, state_vec=False)
    
U = Operator(qc).data
print("\nSize of the U:", U.shape)

H = get_hamiltonian(qc)
print("\nSize of the H:", H.shape)


Size of the U: (1024, 1024)

Size of the H: (1024, 1024)


In [56]:
def hamiltonian_to_pauli(H):
    """Decomposes a Hamiltonian matrix into a sum of Pauli matrices."""
    pauli_op = SparsePauliOp.from_operator(H)
    return pauli_op

# Convert Hamiltonian to a sum of Pauli operators
pauli_string = hamiltonian_to_pauli(H)

with open("hamiltonian_pauli.txt", "w") as f:
    f.write(str(pauli_string))
print("Hamiltonian saved to hamiltonian_pauli.txt")



# Print results
#print("Hamiltonian as a sum of Pauli matrices:\n", pauli_string)

In [128]:
import numpy as np
from qiskit.quantum_info import Statevector

def state_to_hamiltonian(statevector, energy=1):
    """Constructs a Hamiltonian H such that H |ψ⟩ = E |ψ⟩"""
    psi = np.array(statevector)  # Convert to NumPy array
    H = energy * np.outer(psi, psi.conj())  # Compute H = E |ψ⟩⟨ψ|
    return H

# Example: Define a quantum state
#psi = Statevector.from_label("00").data  # Example: |00⟩ state

# Compute the Hamiltonian
H = state_to_hamiltonian(psi_0, energy=1)

# Print the result
print("Hamiltonian Matrix:\n", np.round(H, 4))
print("\nSize of H:", H.shape, "->", H.nbytes, "bytes")

Hamiltonian Matrix:
 [[ 0.0017-0.j     -0.0009+0.0008j  0.0003-0.0009j ...  0.0001+0.0001j
  -0.0015-0.0008j -0.0011+0.0002j]
 [-0.0009-0.0008j  0.0008-0.j     -0.0006+0.0003j ... -0.    -0.0001j
   0.0004+0.0011j  0.0007+0.0004j]
 [ 0.0003+0.0009j -0.0006-0.0003j  0.0005-0.j     ... -0.    +0.0001j
   0.0002-0.0009j -0.0003-0.0005j]
 ...
 [ 0.0001-0.0001j -0.    +0.0001j -0.    -0.0001j ...  0.    +0.j
  -0.0002+0.0001j -0.0001+0.0001j]
 [-0.0015+0.0008j  0.0004-0.0011j  0.0002+0.0009j ... -0.0002-0.0001j
   0.0016-0.j      0.0008-0.0007j]
 [-0.0011-0.0002j  0.0007-0.0004j -0.0003+0.0005j ... -0.0001-0.0001j
   0.0008+0.0007j  0.0007+0.j    ]]

Size of H: (1024, 1024) -> 16777216 bytes


In [129]:
import numpy as np
from qiskit.quantum_info import Statevector

def hamiltonian_to_state(H):
    """Extracts the quantum state from the Hamiltonian."""
    eigenvalues, eigenvectors = np.linalg.eigh(H)  # Diagonalize H
    idx = np.argmax(np.abs(eigenvalues))  # Find the nonzero eigenvalue
    state = eigenvectors[:, idx]  # Extract corresponding eigenvector
    return Statevector(state)  # Convert to a Qiskit statevector

# # Example: Define a state and construct its Hamiltonian
# psi_original = Statevector.from_label("00").data
# H = np.outer(psi_original, psi_original.conj())  # H = |ψ⟩⟨ψ|

# Recover the state from H
psi_reconstructed = hamiltonian_to_state(H)

# Print results
#print("Original Statevector:\n", psi_original)
print("Reconstructed Statevector:\n", psi_reconstructed)


Reconstructed Statevector:
 Statevector([ 0.04129999+0.j        , -0.02169208-0.01852561j,
              0.00714551+0.02190464j, ...,  0.00267192-0.00297484j,
             -0.03512857+0.01988072j, -0.02673348-0.00519205j],
            dims=(2, 2, 2, 2, 2, 2, 2, 2, 2, 2))


In [61]:
import numpy as np
from qiskit.quantum_info import Statevector

def check_states_equal(psi1, psi2):
    """Checks if two quantum states are equal using the inner product."""
    inner_product = np.abs(np.vdot(psi1, psi2))  # Compute inner product |⟨ψ|φ⟩|
    return inner_product

# Check if they are equal
inner_product = check_states_equal(psi_0, psi_reconstructed)

print("Inner Product:", inner_product)

# If inner_product ≈ 1, the states are identical
if np.isclose(inner_product, 1):
    print("The states are equal (up to a global phase).")
else:
    print("The states are NOT equal.")

Inner Product: 0.9999999999999967
The states are equal (up to a global phase).


In [84]:
cat_vec_size = sys.getsizeof(cat_vec)
print(f"Type of cat_vec: {type(cat_vec)}, name: {type(cat_vec).__name__}")
print(f"Dtype of cat_vec: {cat_vec.dtype}")
print(f"Length of cat_vec: {cat_vec.size}")
print(f"Size of cat_vec in bytes: {cat_vec_size}")
print(f"Data size of cat_vec in bytes (nbytes): {cat_vec.nbytes}")

psi_0_vec = np.asarray(psi_0)
psi_0_size = sys.getsizeof(psi_0_vec)
print(f"Type of psi_0_vec: {type(psi_0_vec)}, name: {type(psi_0_vec).__name__}")
print(f"Dtype of psi_0_vec: {psi_0_vec.dtype}")
print(f"Length of psi_0_vec: {psi_0_vec.size}")
print(f"Size of quantum state in bytes: {psi_0_size}")
print(f"Data size of psi_0_vec in bytes (nbytes): {psi_0_vec.nbytes}")


#psi_reconstructed_size = sys.getsizeof(psi_0)
#print(f"Size of quantum state (reconstructed) in bytes: {psi_reconstructed_size}")

Type of cat_vec: <class 'numpy.ndarray'>, name: ndarray
Dtype of cat_vec: float64
Length of cat_vec: 1536
Size of cat_vec in bytes: 12400
Data size of cat_vec in bytes (nbytes): 12288
Type of psi_0_vec: <class 'numpy.ndarray'>, name: ndarray
Dtype of psi_0_vec: complex128
Length of psi_0_vec: 1024
Size of quantum state in bytes: 112
Data size of psi_0_vec in bytes (nbytes): 16384


In [99]:
import numpy as np

def is_hermitian(H):
    """Check if the Hamiltonian is Hermitian (H = H†)."""
    return np.allclose(H, H.conj().T)

def is_unitary(H):
    """Check if the matrix is unitary (H†H = I)."""
    identity = np.eye(H.shape[0])
    return np.allclose(H.conj().T @ H, identity)

# ✅ Check Hermitian property
print("Is H Hermitian?", is_hermitian(H))

# ✅ Check Unitary property (should usually be False for a Hamiltonian)
print("Is H Unitary?", is_unitary(H))

Is H Hermitian? True
Is H Unitary? False


In [97]:
# Decode by applying U†
psi_recovered = U.conj().T @ psi_0_vec

# Reverse amplitude unpacking
psi_recovered = psi_recovered * np.linalg.norm(psi_0_vec)  # scale back
recovered_complex = psi_recovered[:768]
recovered_real = np.empty(1536)
recovered_real[0::2] = recovered_complex.real
recovered_real[1::2] = recovered_complex.imag

# Compare to original
print("Vectors match?", np.allclose(cat_vec, recovered_real, rtol=1e-5, atol=1e-7))


Vectors match? False


In [102]:

U = Operator(qc).data

print("Unitary?", np.allclose(U.conj().T @ U, np.eye(2)))

NameError: name 'circuit' is not defined

In [6]:
def pad_vector(vector):
    """
    Pads the input complex vector with zeros so that its length is the next power of 2.
    """
    n = len(vector)
    next_power = 2 ** int(np.ceil(np.log2(n)))
    if n == next_power:
        return vector
    padded = np.zeros(next_power, dtype=complex)
    padded[:n] = vector
    return padded

def encode_to_qubit(arr):
    """
    Direct amplitude encoding:
    Groups pairs of reals into complex numbers, pads to next power of 2, and normalizes.
    """
    # Pair the real numbers into complex amplitudes.
    complex_vec = pair_real_to_complex(arr)
    # Pad the vector to the next power of 2.
    complex_vec = pad_vector(complex_vec)
    norm = np.linalg.norm(complex_vec)
    return complex_vec / norm if norm != 0 else complex_vec

def rotation_matrix_y(theta):
    """Construct an R_y(theta) rotation matrix."""
    return np.array([
        [np.cos(theta/2), -np.sin(theta/2)],
        [np.sin(theta/2),  np.cos(theta/2)]
    ], dtype=complex)

def rotation_matrix_z(phi):
    """Construct an R_z(phi) phase shift matrix."""
    return np.array([
        [np.exp(-1j * phi/2), 0],
        [0, np.exp(1j * phi/2)]
    ], dtype=complex)

def recursive_state_preparation(psi):
    """
    Recursively prepares a state |psi> (with 2^n amplitudes) from |0...0>
    by splitting the state into left and right halves. At each step,
    we compute the rotation angle theta and relative phase phi so that
      cos(theta/2) = norm(left branch)
      sin(theta/2) = norm(right branch)
    and we force the left branch amplitude to be real and nonnegative.
    
    The unitary on the current qubit is defined as:
      U' = exp(i*phi/2)*R_z(phi)*R_y(theta)
    so that U'|0> = [cos(theta/2), sin(theta/2)*exp(i*phi)].
    """
    # Force global phase so that psi[0] is real and nonnegative.
    if psi[0] != 0:
        psi = psi * np.exp(-1j * np.angle(psi[0]))
    
    n = int(np.log2(len(psi)))
    if n == 0:
        return psi

    half = len(psi) // 2
    norm_left = np.linalg.norm(psi[:half])
    norm_right = np.linalg.norm(psi[half:])
    
    # Compute rotation angle: cos(theta/2)=norm_left, sin(theta/2)=norm_right.
    theta = 2 * np.arctan2(norm_right, norm_left)
    
    # Set the relative phase from the right branch.
    phi = np.angle(psi[half]) if (norm_left > 0 and norm_right > 0) else 0.0

    # Build the corrected single-qubit unitary.
    U = np.exp(1j * phi/2) * (rotation_matrix_z(phi) @ rotation_matrix_y(theta))
    qubit_state = U @ np.array([1, 0], dtype=complex)
    
    # Recursively prepare the left and right branches.
    if norm_left > 0:
        psi_left = psi[:half] / norm_left
        left_state = recursive_state_preparation(psi_left)
    else:
        left_state = np.zeros(half, dtype=complex)
    if norm_right > 0:
        psi_right = psi[half:] / norm_right
        right_state = recursive_state_preparation(psi_right)
    else:
        right_state = np.zeros(half, dtype=complex)
    
    return np.concatenate([qubit_state[0] * left_state,
                           qubit_state[1] * right_state])

def amplitude_encoding_real_hardware(vector):
    """
    Simulates amplitude encoding using only unitary operations
    (as on real quantum hardware) by recursively preparing the state.
    """
    # Pair the real vector into complex amplitudes and pad to a power of 2.
    complex_vec = pair_real_to_complex(vector)
    complex_vec = pad_vector(complex_vec)
    
    norm = np.linalg.norm(complex_vec)
    if norm == 0:
        raise ValueError("Vector cannot be zero.")
    target_state = complex_vec / norm

    # Force global phase so that the first amplitude is real and nonnegative.
    if target_state[0] != 0:
        target_state = target_state * np.exp(-1j * np.angle(target_state[0]))
    
    # Prepare the state recursively.
    return recursive_state_preparation(target_state)

def fix_global_phase(state):
    """Force the first amplitude to be real and nonnegative."""
    if state[0] != 0:
        return state * np.exp(-1j * np.angle(state[0]))
    return state

# --- Use the 'cat' embedding ---

stored_embeddings = load_embeddings()
cat_vec = get_embedding("cat", stored_embeddings)
if cat_vec is None:
    print("No embedding found for 'cat'")
else:
    cat_vec = np.array(cat_vec)  # Should be of length 1536 (float64)

    # Direct amplitude encoding on cat_vec.
    cat_state_direct = fix_global_phase(encode_to_qubit(cat_vec))
    print("Direct encoding state vector length:", len(cat_state_direct))
    
    # Simulated hardware state preparation on cat_vec.
    cat_state_hardware = fix_global_phase(amplitude_encoding_real_hardware(cat_vec))
    print("Hardware-simulated state vector length:", len(cat_state_hardware))
    
    # Optionally, force the same global phase.
    phase_offset = np.angle(inner_product(cat_state_direct, cat_state_hardware))
    cat_state_hardware *= np.exp(-1j * phase_offset)
    
    # Compute the inner product.
    comparison_inner_prod = inner_product(cat_state_direct, cat_state_hardware)
    comparison_magnitude = np.abs(comparison_inner_prod)
    
    print("Inner Product Magnitude Between Methods:", comparison_magnitude)
    
    if np.isclose(comparison_magnitude, 1.0, atol=1e-6):
        print("The two methods produce equivalent quantum states (up to a global phase).")
    else:
        print("The two methods produce different quantum states.")


NameError: name 'get_embedding' is not defined

In [33]:
from qiskit.providers.basic_provider import BasicSimulator

def pad_real_vector_for_complex_encoding(vec):
    """Pad real vector to the next multiple of 2 that gives 2^n complex amplitudes."""
    if len(vec) % 2 != 0:
        vec = np.append(vec, 0.0)  # make even length
    complex_len = len(vec) // 2
    next_pow2 = 2 ** int(np.ceil(np.log2(complex_len)))
    target_real_len = 2 * next_pow2
    return np.pad(vec, (0, target_real_len - len(vec)))
    
def swap_test_statevectors(state1, state2):
    n = int(np.log2(len(state1)))

    anc = QuantumRegister(1, 'anc')
    qr1 = QuantumRegister(n, 'reg1')
    qr2 = QuantumRegister(n, 'reg2')
    cr = ClassicalRegister(1, 'c')
    qc = QuantumCircuit(anc, qr1, qr2, cr)

    qc.h(anc[0])
    for i in range(n):
        qc.cswap(anc[0], qr1[i], qr2[i])
    qc.h(anc[0])
    qc.measure(anc[0], cr[0])

    # Combine the full statevector manually (ancilla last!)
    full_vec = Statevector.from_label('0').tensor(state1).tensor(state2)  # order: anc ⊗ ψ ⊗ φ

    print("Ancilla length:", len(Statevector.from_label('0')))
    print("State1 length:", len(state1))
    print("State2 length:", len(state2))
    
    # Initialize the circuit into this state
    qc.initialize(full_vec.data, qc.qubits)

    simulator = BasicSimulator()

    # Simulate the circuit
    compiled_circuit = transpile(qc, simulator)
    job = simulator.run(compiled_circuit, shots=1024)
    result = job.result()
    counts = result.get_counts()
    
    # Print the counts
    print(result.get_counts())

    p0 = counts.get('0', 0) / 1024
    similarity_squared = 2 * p0 - 1
    similarity = np.sqrt(max(0, similarity_squared))
    return similarity, counts

cat_vec_padded = pad_real_vector_for_complex_encoding(cat_vec)
dog_vec_padded = pad_real_vector_for_complex_encoding(dog_vec)
car_vec_padded = pad_real_vector_for_complex_encoding(car_vec)

psi_cat = Statevector(fix_global_phase(amplitude_encoding_real_hardware(cat_vec_padded)))
psi_dog = Statevector(fix_global_phase(amplitude_encoding_real_hardware(dog_vec_padded)))
psi_car = Statevector(fix_global_phase(amplitude_encoding_real_hardware(car_vec_padded)))

similarity, counts = swap_test_statevectors(psi_cat, psi_dog)
print("Estimated similarity, cat<>dog:", similarity)
print("Measurement counts:", counts)

similarity, counts = swap_test_statevectors(psi_cat, psi_car)
print("Estimated similarity, cat<>car:", similarity)
print("Measurement counts:", counts)


Ancilla length: 2
State1 length: 1024
State2 length: 1024


KeyboardInterrupt: 